In [35]:
#This file reads the note_csv, spectrums. Generate and save the numpy arrays which are inputs to the model

In [36]:
import numpy as np
import glob

In [44]:
spectrums = 'E:/notes_database/spectrograms/'
files = glob.glob('E:/notes_database/index/*')

In [38]:
def parse_row(csv_row):  # same as generate_database.ipynb

    csv_row_array = csv_row.split(",")

    filename = csv_row_array[0]
    time_array = [float(t) for t in csv_row_array[1::2]]
    note_array = [int(x) for x in csv_row_array[2::2]]

    assert len(time_array) == len(note_array)

#     print(time_array)
#     print(note_array)

    return filename, time_array, note_array

In [39]:
def parse_csv(csv_file): # Convert the csv files into numpy arrays
    
    with open(csv_file, 'r') as csvfile:
        csv_rows = csvfile.read().split("\n")

    filenames = []
    time_arrays = []
    note_arrays = []
    
    for csv_row in csv_rows:
        filename, time_array, note_array = parse_row(csv_row)
        
        filenames.append(filename)
        time_arrays.append(time_array)
        note_arrays.append(note_array)
    
    return filenames,time_arrays,note_arrays

In [40]:
parse_csv(files[0])

(['F88A365E5D3B94DD',
  '1934D82339D7C8CA',
  'A00720532DE9C614',
  '7C531CA1BA2156B3',
  'ACF7C06411F8B320'],
 [[0.562,
   0.562,
   0.562,
   0.562,
   1.326,
   1.326,
   1.326,
   2.245,
   2.245,
   2.245,
   2.928,
   2.928,
   2.928,
   3.956,
   3.956,
   5.224,
   5.224,
   5.224,
   5.224,
   6.315,
   6.315,
   6.315,
   7.355,
   8.201,
   8.201,
   8.201,
   9.282,
   9.282,
   9.282,
   9.803,
   9.803,
   9.803,
   11.081,
   11.081,
   11.081,
   12.136,
   12.136,
   12.136,
   12.136,
   13.007,
   13.007,
   13.007,
   13.007,
   14.219,
   14.219,
   15.272,
   15.272,
   15.272,
   15.272,
   15.967,
   15.967,
   15.967,
   16.96,
   16.96,
   16.96,
   17.704,
   17.704,
   17.704,
   18.643,
   18.643,
   19.416,
   19.416,
   20.208,
   20.208,
   20.208],
  [1.094,
   1.094,
   1.094,
   1.094,
   1.736,
   1.736,
   2.691,
   2.691,
   3.206,
   3.206,
   3.206,
   3.741,
   3.741,
   3.741,
   3.741,
   5.02,
   5.02,
   5.02,
   6.082,
   6.082,
   6.082,
 

In [83]:
def prepare_inputs(csv_name):
    # the time for a unit width of spectrum is about 0.003 second
    # however a unit in time_array is 0.5 second, so to calculate the unit numbers, should divide the time_number by 0.006
    filenames,time,note = parse_csv(csv_name)
    lstm_all_inputs = []
    cnn_all_inputs = []
    y_all_train = []
    for file in range(len(filenames)):
        one_row = np.zeros(24)
        previous_time = [time[file][0]]
        spectrum = np.load(spectrums+filenames[file]+'.npy')
        y_train = []
        # load the spectrum numpy array from 20 units before the timing(onset) to 88 units after the timing
        for i in range(len(time[file])):
            if time[file][i] == previous_time[-1]:
                one_row[note[file][i]] = 1
            else:
                #append lstm input
                lstm_input = np.zeros((5,24))
                for num in range(len(previous_time)):
                    # (5(steps) * 81(4/5 108 units, overlapped by 1/5) + 20)* 0.006 = 2.55
                    if previous_time[-1] - previous_time[num] < 2.55:
                        checkpoint = num
                        break
                        
                start_time = previous_time[-1] - (81*5 + 20)*0.006
                if start_time < 0:
                    start_time = 0
                
                for num in range(checkpoint, len(y_train)):
                    position = int(((previous_time[num]-start_time)//0.006)//81) 
                    lstm_input[position] = y_train[num]
                lstm_all_inputs.append(lstm_input)
                
                #append y_train
                count_ = 0
                for ele in one_row:
                    if ele == 1:
                        count_ += 1
                if count_ > 4:
                    print(previous_time, one_row)
                y_train.append(one_row)
                y_all_train.append(one_row)
                
                #append cnn input
                cnn_input = spectrum[:, int(previous_time[-1]//0.006-20):int(previous_time[-1]//0.006+88)]
                cnn_all_inputs.append(cnn_input)
                
                one_row = np.zeros(24)
                one_row[note[file][i]] = 1
                previous_time.append(time[file][i])
            
            if i == len(time[file])-1:
                lstm_input = np.zeros((5,24))
                for num in range(len(previous_time)):
                    # (5(steps) * 81(4/5 108 units, overlapped by 1/5) + 20) * 0.006 = 2.55
                    if previous_time[-1] - previous_time[num] < 2.55:
                        checkpoint = num
                        break
                        
                start_time = previous_time[-1] - (81*5 + 20)*0.006
                if start_time < 0:
                    start_time = 0
                
                for num in range(checkpoint, len(y_train)):
                    position = int(((previous_time[num]-start_time)//0.006)//81)
                    lstm_input[position] = y_train[num]
                lstm_all_inputs.append(lstm_input)
                
                #append y_train
                count_ = 0
                for ele in one_row:
                    if ele == 1:
                        count_ += 1
                if count_ > 4:
                    print(previous_time, one_row)
                y_train.append(one_row)
                y_all_train.append(one_row)
                
                #append cnn input
                cnn_input = spectrum[:, int(previous_time[-1]//0.006-20):int(previous_time[-1]//0.006+88)]
                cnn_all_inputs.append(cnn_input)
                
        #check if the last cnn_input has right shape
        if cnn_all_inputs[-1].shape != (108, 108):
            cnn_all_inputs = cnn_all_inputs[:-1]
            lstm_all_inputs = lstm_all_inputs[:-1]
            y_all_train = y_all_train[:-1]
            
    lstm_all_inputs = np.array(lstm_all_inputs)
    cnn_all_inputs = np.array(cnn_all_inputs).reshape(len(cnn_all_inputs), 108, 108, 1)
    y_all_train = np.array(y_all_train)
    return lstm_all_inputs, cnn_all_inputs, y_all_train

In [42]:
save = 'E:/notes_database/'

In [84]:
count = 0
number = 0
lstm_save, cnn_save, y_train_save = prepare_inputs(files[0])
for i in range(len(files))[1:]:
    if number < 5:
        lstm, cnn, y_train = prepare_inputs(files[i])
        lstm_save = np.append(lstm_save, lstm, axis=0)
        cnn_save = np.append(cnn_save, cnn, axis=0)
        y_train_save = np.append(y_train_save, y_train, axis=0)
        number += 1
    elif number >= 5:
        np.save(save+'lstm_inputs/{}.npy'.format(count), lstm_save.astype(np.float16))
        np.save(save+'cnn_inputs/{}.npy'.format(count), cnn_save.astype(np.float16))
        np.save(save+'y_train/{}.npy'.format(count), y_train_save.astype(np.float16))
        count += 1
        number = 1
        lstm_save, cnn_save, y_train_save = prepare_inputs(files[i])
        
np.save(save+'lstm_inputs/{}.npy'.format(count), lstm_save.astype(np.float16))
np.save(save+'cnn_inputs/{}.npy'.format(count), cnn_save.astype(np.float16))
np.save(save+'y_train/{}.npy'.format(count), y_train_save.astype(np.float16))